In [1]:
import sys
import MeCab
m = MeCab.Tagger ("mecab-ipadic-neologd")

In [2]:
def parse_mecab(block):
    res = []
    
    for line in block.split('\n'):
        if line == '':
            return res
        (surface, node) = line.split('\t')
        pos,pos1,_,_,_,_,base,*_ = node.split(",")
        lineDict = {
            'surface': surface,
            'base': base,
            'pos': pos,
            'pos1': pos1
        }
        if surface == "\u3000":
            continue
        res.append(lineDict)

In [3]:
with open("neko.txt.mecab",mode="rt",encoding="utf-8") as f:
    txt = f.read().split("EOS\n")

In [4]:
txt = list(filter(lambda x: x!= "",txt))
doc = [parse_mecab(block) for block in txt if len(block) != 0]

In [5]:
doc = [i for i in doc if i != []]

doc = []

for i in txt:
    node = m.parseToNode(i)
    sent = []
    while node:
        d = dict()
        surface = node.surface
        if len(node.feature.split(",")) == 6:
            pos,pos1,*_ = node.feature.split(",")
            base = pos
        else:
            pos,pos1,_,_,_,_,_base,*_ = node.feature.split(",")
        d["surface"] = surface
        d["pos"] = pos
        d["pos1"] = pos1
        d["base"]= base
        sent.append(d)
        node = node.next
    doc.append(sent)

with open(fname_parsed) as file_parsed:

    morphemes = []
    for line in file_parsed:

        # 表層形はtab区切り、それ以外は','区切りでバラす
        cols = line.split('\t')
        if(len(cols) < 2):
            raise StopIteration     # 区切りがなければ終了
        res_cols = cols[1].split(',')

        # 辞書作成、リストに追加
        morpheme = {
            'surface': cols[0],
            'base': res_cols[6],
            'pos': res_cols[0],
            'pos1': res_cols[1]
        }
        morphemes.append(morpheme)

        # 品詞細分類1が'句点'なら文の終わりと判定
        if res_cols[1] == '句点':
            yield morphemes
            morphemes = []

txt = txt.replace("\u3000","")

txt = [i for i in txt.split("\n") if len(i) > 1]

doc = []

for i in txt:
    node = m.parseToNode(i)
    sent = []
    while node:
        d = dict()
        surface = node.surface
        if len(node.feature.split(",")) == 6:
            pos,pos1,*_ = node.feature.split(",")
            base = pos
        else:
            pos,pos1,_,_,_,_,_base,*_ = node.feature.split(",")
        d["surface"] = surface
        d["pos"] = pos
        d["pos1"] = pos1
        d["base"]= base
        sent.append(d)
        node = node.next
    doc.append(sent)

In [6]:
doc[:100]

[[{'surface': '一', 'base': '一', 'pos': '名詞', 'pos1': '数'}],
 [{'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'},
  {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'},
  {'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'},
  {'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'},
  {'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'},
  {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}],
 [{'surface': '名前', 'base': '名前', 'pos': '名詞', 'pos1': '一般'},
  {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'},
  {'surface': 'まだ', 'base': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続'},
  {'surface': '無い', 'base': '無い', 'pos': '形容詞', 'pos1': '自立'},
  {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}],
 [{'surface': 'どこ', 'base': 'どこ', 'pos': '名詞', 'pos1': '代名詞'},
  {'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'},
  {'surface': '生れ', 'base': '生れる', 'pos': '動詞', 'pos1': '自立'},
  {'surface': 'た', 'base': 'た', 'pos': '助動詞', 'pos1': '*'},
  {'surface': 

In [11]:
noun_phrase = []
for i in doc:
    for j in range(len(i)-2):
        if i[j]["pos"] == "名詞" and i[j+1]["base"] == "の" and i[j+2]["pos"] == "名詞":
            tmp = i[j]["surface"] + i[j+1]["surface"] + i[j+2]["surface"]
            noun_phrase.append(tmp)

In [12]:
noun_phrase[:100]

['彼の掌',
 '掌の上',
 '書生の顔',
 'はずの顔',
 '顔の真中',
 '穴の中',
 '書生の掌',
 '掌の裏',
 '何の事',
 '肝心の母親',
 '藁の上',
 '笹原の中',
 '池の前',
 '池の上',
 '一樹の蔭',
 '垣根の穴',
 '隣家の三',
 '時の通路',
 '一刻の猶予',
 '家の内',
 '彼の書生',
 '以外の人間',
 '前の書生',
 'おさんの隙',
 'おさんの三',
 '胸の痞',
 '家の主人',
 '主人の方',
 '鼻の下',
 '吾輩の顔',
 '自分の住',
 '吾輩の主人',
 '家のもの',
 'うちのもの',
 '彼の書斎',
 '本の上',
 '皮膚の色',
 '本の上',
 '彼の毎夜',
 '以外のもの',
 '主人の傍',
 '彼の膝',
 '膝の上',
 '経験の上',
 '飯櫃の上',
 '炬燵の上',
 'ここのうち',
 '供の寝床',
 '彼等の中間',
 '供の一',
 '例の神経',
 '性の主人',
 '次の部屋',
 '自分の勝手',
 '吾輩の方',
 '台所の板の間',
 '吾輩の尊敬',
 '向の白',
 '玉のよう',
 'そこの家',
 '家の書生',
 '裏の池',
 '親子の愛',
 'もっともの議論',
 '刺の頭',
 '鰡の臍',
 '彼等のため',
 '軍人の家',
 '代言の主人',
 '教師の家',
 '猫の時節',
 '吾輩の家',
 '家の主人',
 'だらけの英文',
 '胃弱の癖',
 '後架の中',
 '平の宗',
 '月の月給',
 '当分の間',
 '下のよう',
 '今更のよう',
 '主人の述懐',
 '彼の友',
 '金縁の眼鏡',
 '主人の顔',
 '内の想像',
 '訳のもの',
 '利の大家',
 '金縁の裏',
 '吾輩の後ろ',
 '彼の友',
 '吾輩の輪廓',
 '顔のあたり',
 '上乗の出来',
 '顔の造作',
 '他の猫',
 '不器量の吾輩',
 '吾輩の主人',
 '斯産の猫',
 '斑入りの皮膚']